### Trade Demo

#### Goal: 
- Load the trade data for the country `Canada`
- Launch a domain node for canada
- Login into the domain node
- Upload `Canada's` trade on the domain node
- Create a Data Scientist User
- Accept/Decline any requests from the DS

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

canada = pd.read_csv("../datasets/ca - feb 2021.csv")

/Users/andrewliamtrask/opt/anaconda3/envs/syft39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Step 1: Load the dataset

We have trade data for the country, which has provided data from Feb 2021. They key colums are:

- Commodity Code: the official code of that type of good
- Reporter: the country claiming the import/export value
- Partner: the country being claimed about
- Trade Flow: the direction of the goods being reported about (imports, exports, etc)
- Trade Value (US$): the declared USD value of the good

Let's have a quick look at the top five rows of the dataset.

In [2]:
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


### Step 2: Spin up the Domain Node

As the main requirement of this demo is to perform analysis on the Canada's trade dataset. So, we need to spin up a domain node for Canada.

Assuming you have [Docker](https://www.docker.com/) installed and configured with >=8GB of RAM, navigate to PySyft/packages/hagrid and run the following commands in separate terminals (can be done at the same time):


```bash
# install hagrid cli tool
pip install -e .
```

```bash
hagrid launch Canada domain
```

<div class="alert alert-block alert-info">
    <b>Quick Tip:</b> Don't run this now, but later when you want to stop these nodes, you can simply run the same argument with the "stop" command. So from the PySyft/grid directory you would run. Note that these commands will delete the database by default. Add the flag "--keep_db=True" to keep the database around. Also note that simply killing the thread created by ./start is often insufficient to actually stop all nodes. Run the ./stop script instead. To stop the nodes listed above (and delete their databases) run:

```bash
hagrid land Canada
```
</div>

### Step 3: Login into the Domain

In [4]:
import syft as sy

# Let's login into the domain node
domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081...

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


### Step 4: Upload Canada's trade data on the domain

In [5]:
# Great !! we're logged in. Let's upload the dataset.
# For, simplicity we will upload the first 10000 rows of the dataset.

domain_node.load_dataset(
    assets={"feb2020": canada[:10000]},
    name="Canada Trade Data - First 10000 rows",
    description="""A collection of reports from Canada's statistics 
                    bureau about how much it thinks it imports and exports from other countries.""",
)

Cool !!! The dataset was successfully uploaded onto the domain.

In [6]:
# Now, let's check datasets available on the domain.
domain_node.datasets.pandas()

,name,blob_metadata,data,description,manifest,tags,id,str_metadata
0,b'\n\x16syft.lib.python.String\x129\n!iStat Tr...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,b'\n\x19syft.lib.python.list.List\x12\xa5\x04\...,"b""\n\x16syft.lib.python.String\x12\xaa\x01\n\x...",b'\n\x16syft.lib.python.String\x12\x16\x12\x12...,b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,b'\n\x16syft.lib.python.String\x12<\n$9a4c698b...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...
1,b'\n\x16syft.lib.python.String\x12<\n$Canada T...,b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...,"b""\n\x19syft.lib.python.list.List\x12\xa5\x04\...","b""\n\x16syft.lib.python.String\x12\xab\x01\n\x...",b'\n\x16syft.lib.python.String\x12\x16\x12\x12...,b'\n\x19syft.lib.python.list.List\x12\x16\x12\...,"b""\n\x16syft.lib.python.String\x12<\n$9cb8ca41...",b'\n\x14syft.lib.python.Dict\x12\x14\x1a\x12\n...


### Step 5: Create a Data Scientist User

In [7]:
# Now, let's create a user on the domain. This user will act as the Data Scientist.
# Let's assume the Data Scientist, has provided their name and email.
domain_node.users.create(
    name="Sheldon Cooper",
    email="sheldon@caltech.edu",
    password="bazinga"
)

```
Great !!! We were successfully able to create a new user. 
Now, let's move to the Data Scientist notebook, to check out their experience.
```

### Step 6: Accept/Decline Requests

In [8]:
# Let's check if there are any requests pending for approval.
domain_node.requests.pandas

""


In [81]:
# Looks like the DS wants to download the whole dataset. We cannot allow that.
# Let's select and deny this request.
domain_node.requests[0].deny()

In [9]:
# Now, if we check there are no more pending requests.
domain_node.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[#feb2020],Access whole dataset,<UID: 096a522892764fa09ab48dd47f999b57>,<UID: 6433dbda46944d85ad83a3a0f75ea4ab>,<class 'pandas.core.frame.DataFrame'>
1,"[#feb2020, __getitem__, __eq__, __and__, __get...",Goods value imported from Egypt,<UID: 3883236a3ff9474593252b0f96a1b3c3>,<UID: eac75a56e5ca4786a59b5614f17a9c08>,
2,"[#feb2020, __getitem__, __eq__, __and__, __get...",Goods value imported from Egypt,<UID: d3ade217b12f41e587ea883c4709e4d5>,<UID: ca17e4bc69b94513b8a03fab44064127>,
3,"[#feb2020, __getitem__, __eq__, __and__, __and...",Goods value exported to Egypt,<UID: c2f6dbe2ee4049a9bd089450e8aaf315>,<UID: 976a957ddf87401f8eb5f46cffb1814b>,


In [69]:
# Some time has passed..... Let's check again
domain_node.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,"[#feb2020, __getitem__, __eq__, __and__, __get...",Goods value imported from Egypt,<UID: 4c9f1801a0714f9f9b4b25c80718c071>,<UID: d817286b6fd6416f8136842cd19b6ffe>,
1,"[#feb2020, __getitem__, __eq__, __and__, __get...",Goods value imported from Egypt,<UID: c0385c69bb7244778fa157c269547eee>,<UID: eb5bda2dd2b7486780012c0292b0b750>,


In [70]:
# Let's select the request and approve it.
domain_node.requests[0].accept()
domain_node.requests[1].accept()

In [71]:
# Now if we check the request queue, it should be empty.
domain_node.requests.pandas

""


In [77]:
# Let's check again.....
domain_node.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,"[#feb2020, __getitem__, __eq__, __and__, __and...",Goods value exported to Egypt,<UID: 14cbbada0dc64e21848d7a3878280323>,<UID: d78aebb6c1e74193b3d90d4884cfcac0>,
1,"[#feb2020, __getitem__, __eq__, __and__, __and...",Goods value Re-Exported to Egypt,<UID: 2ad25af2892d4ba9b4ebef2c5bfb3a81>,<UID: 9ca3ea460e3649b090b3ca831b2e7de2>,


In [78]:
# There are two more requests
domain_node.requests[0].approve()
domain_node.requests[1].approve()